In [1]:
import torch
import os
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from echotorch.nn import ESN, LiESN
from echotorch.utils.matrix_generation import *
import numpy as np
from scipy.special import softmax
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
import torchvision
import pathlib
import librosa
import librosa.display
from statistics import mean, stdev

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [2]:
data_path = '../Hyperdimensional/Datasets/Unprocessed/Chorales/dataset'
img_path = '../HyperDimensional/img_data'

In [3]:
def process_images():
	cmap = plt.get_cmap('inferno')
	plt.figure(figsize=(8,8))
	pathlib.Path(f'{img_path}').mkdir(parents=True, exist_ok=True)

	for filename in tqdm(os.listdir(f'{data_path}/train/')):
	    songname = f'{data_path}/train/{filename}'
	    y, sr = librosa.load(songname, mono=True, duration=60)
	    plt.specgram(y, NFFT=2048, Fs=2, Fc=0, cmap=cmap, sides='default', mode='default', scale='dB')
	    plt.axis('off')
	    figfile = f'{img_path}/train/Chorales/{filename[:-3].replace(".", "")}.png'
	    plt.savefig(figfile)
	    plt.clf()

	for filename in tqdm(os.listdir(f'{data_path}/test/')):
	    songname = f'{data_path}/test/{filename}'
	    y, sr = librosa.load(songname, mono=True, duration=60)
	    plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB')
	    plt.axis('off')
	    figfile = f'{img_path}/test/Chorales/{filename[:-3].replace(".", "")}.png'
	    plt.savefig(figfile)
	    plt.clf()
		
	# for filename in tqdm(os.listdir(f'{data_path}/test/')):
	#     songname = f'{data_path}/test/{filename}'
	#     y, sr = librosa.load(songname, mono=True, duration=60)
	#     S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
	#     librosa.display.specshow(librosa.power_to_db(
	# 	                            S, 
	#                                 ref=np.max), 
	#                                 y_axis='mel', 
	#                                 fmax=8000,
	# 	                            x_axis='time')
	#     plt.tight_layout()
	#     plt.axis('off')
	#     figfile = f'{img_path}/mel/{filename[:-3].replace(".", "")}.png'
	#     plt.savefig(figfile)
	#     plt.clf()

In [4]:
# process_images()

In [5]:
batch_size = 32
im_size = 512
CHORALE_IDX = 0
NCHORALE_IDX = 1
TRAIN_COUNT = len(os.listdir(f'{img_path}/train/Chorales/'))
TEST_COUNT = len(os.listdir(f'{img_path}/test/Chorales/'))

In [6]:
class MImageFolder(torchvision.datasets.ImageFolder):
	def __init__(self, root, transform=None):
		super(MImageFolder, self).__init__(root=root, transform=transform)
		samples = [(img, img.split('\\')[2].split('.')[0]) for img, _ in self.samples]
		self.samples = samples

def normalization_parameter(dataloader):
    mean = 0
    std = 0
    nb_samples = len(dataloader.dataset)
    for data, _ in tqdm(dataloader):
        batch_samples = data.size(0)
        data = data.view(batch_samples, data.size(1), -1)
        mean += data.mean(2).sum(0)
        std += data.std(2).sum(0)
    mean /= nb_samples
    std /= nb_samples
    return mean.numpy(),std.numpy()

im_transforms = transforms.Compose([transforms.ToTensor()])
train_data = MImageFolder(root=f'{img_path}/train', transform=im_transforms)
train_loader = DataLoader(train_data, batch_size=batch_size , shuffle=True)
mean,std = normalization_parameter(train_loader)

  0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
im_transforms = transforms.Compose([
								transforms.Resize((im_size,im_size)),
							    # transforms.RandomResizedCrop(size=315, scale=(0.95, 1.0)),
							    # transforms.RandomRotation(degrees=10),
							    # transforms.RandomHorizontalFlip(),
							    # transforms.CenterCrop(size=299),
							    transforms.ToTensor(),
							    transforms.Normalize(mean,std)])


train_dataset = MImageFolder(root=f'{img_path}/train', transform=im_transforms)
# train_size = int(0.9 * len(train_dataset))
# train_dataset, val_dataset = random_split(train_dataset, [train_size, len(train_dataset) - train_size])
test_dataset = MImageFolder(root=f'{img_path}/test', transform=im_transforms)

train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
# val_loader = DataLoader(train_dataset, batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, 1, shuffle=True)

In [8]:
def create_train_set():
	loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

	# test_imgs = torch.empty((277, 3*im_size*im_size))
	# test_labels = torch.empty((277, 2))
	# 
	# for batch_num, (data, label) in enumerate(test_loader):
	# 	img = torch.flatten(data).to(torch.float)
	# 
	# 	test_imgs[batch_num] = img
	# 	test_labels[batch_num] = torch.zeros(2)

	train_imgs = torch.empty((TRAIN_COUNT, 3*im_size*im_size))
	# train_labels = torch.empty((TRAIN_COUNT, 2))
	train_labels = torch.empty(TRAIN_COUNT)

	for batch_num, (data, label) in enumerate(loader):
		img = torch.flatten(data).to(torch.float)

		train_imgs[batch_num] = img
		# train_labels[batch_num] = torch.zeros(2)
		train_labels[batch_num] = torch.zeros(1)
		if label[0].startswith('random'):
			# train_labels[batch_num][NCHORALE_IDX] = torch.tensor(1).to(torch.float)
			train_labels[batch_num] = torch.tensor(0).to(torch.float)
		else:
			# train_labels[batch_num][CHORALE_IDX] = torch.tensor(1).to(torch.float)
			train_labels[batch_num] = torch.tensor(1).to(torch.float)

	return train_imgs, train_labels

In [9]:
def sigmoid(x):
	return 1 / (1 + np.exp(-x))

def test_model(model, loader):
	results = {}

	for num, (img, label) in enumerate(loader):
		result = model(img.view(1, -1, 3*im_size*im_size).to(torch.float).to(device))
		prediction = result.cpu().numpy().flatten()
		results[label[0]] = sigmoid(prediction)
		# results[label[0]] = prediction
		# results[label[0]][0] = sigmoid(results[label[0]][0])
		# results[label[0]][1] = sigmoid(results[label[0]][1])

	return results

In [10]:
def train_model(res_size):
	torch.cuda.empty_cache()
	esn = LiESN(
		input_dim=3*im_size*im_size,
		hidden_dim=res_size,
		output_dim=1,
		w_generator=UniformMatrixGenerator(),
		wbias_generator=UniformMatrixGenerator(),
		win_generator=UniformMatrixGenerator(),
		learning_algo='pinv',
		nonlin_func=torch.sigmoid,
		leaky_rate=0.9,
		ridge_param=0.25
	).to(device)

	# for input, label in train_loader:
	input = train_imgs.view(1, -1, 3*im_size*im_size).to(torch.float).to(device)
	label = train_labels.view(1, -1, 1).to(torch.float).to(device)
	esn(input, label)
	return esn

In [11]:
def print_scores(results):
	mean_scores = {'boring': 0, 'random': 0, 'comp': 0, 'chorale': 0}
	totals = {'boring': 0, 'random': 0, 'comp': 0, 'chorale': 0}
	for (file, score) in results.items():
		if file.startswith('rand'):
			totals['random'] += 1
			mean_scores['random'] += score.item()
		if file.startswith('boring'):
			totals['boring'] += 1
			mean_scores['boring'] += score.item()
		if file.startswith('comp'):
			totals['comp'] += 1
			mean_scores['comp'] += score.item()
		else:
			totals['chorale'] += 1
			mean_scores['chorale'] += score.item()
	
	for key in ['boring', 'random', 'comp', 'chorale']:
		mean_scores[key] /= totals[key]
	
	if totals['boring'] > 0:
		print(f"Boring: {round(mean_scores['boring'],5)}")
	if totals['random'] > 0:
		print(f"Random: {round(mean_scores['random'],5)}")
	if totals['comp'] > 0:
		print(f"Composed: {round(mean_scores['comp'],5)}")
	if totals['chorale'] > 0:
		print(f"Chorale: {round(mean_scores['chorale'],5)}")

	return mean_scores

def print_results(results):
	predictions = {}
	for (file, result) in results.items():
		predictions[file] = softmax(result).flatten()
		predictions[file] = 'Chorale' if np.argmax(predictions[file], 0) == CHORALE_IDX else 'Not chorale'

	accuracy = {'boring': 0, 'random': 0, 'comp': 0, 'chorale': 0}
	totals = {'boring': 0, 'random': 0, 'comp': 0, 'chorale': 0}
	for (file, pred) in predictions.items():
		if file.startswith('rand'):
			totals['random'] += 1
			accuracy['random'] += pred == 'Not chorale'
		if file.startswith('boring'):
			totals['boring'] += 1
			accuracy['boring'] += pred == 'Not chorale'
		if file.startswith('comp'):
			totals['comp'] += 1
			accuracy['comp'] += pred == 'Chorale'
		else:
			totals['chorale'] += 1
			accuracy['chorale'] += pred == 'Chorale'

	if totals['boring'] > 0:
		print(f"Classified {accuracy['boring']}/{totals['boring']} ({round(100*accuracy['boring']/totals['boring'],2)}%) boring compositions as not chorales")
	if totals['random'] > 0:
		print(f"Classified {accuracy['random']}/{totals['random']} ({round(100*accuracy['random']/totals['random'],2)}%) random compositions as not chorales")
	if totals['comp'] > 0:
		print(f"Classified {accuracy['comp']}/{totals['comp']} ({round(100*accuracy['comp']/totals['comp'],2)}%) composition as chorales")
	if totals['chorale'] > 0:
		print(f"Classified {accuracy['chorale']}/{totals['chorale']} ({round(100*accuracy['chorale']/totals['chorale'],2)}%) dataset chorales as chorales")

	return accuracy, totals

In [12]:
SIZES = [750]
ALPHA = [0.9]

stats = {}
train_imgs, train_labels = create_train_set()

scores = []

for RUN in trange(10):
	for res_size in tqdm(SIZES):
		for alpha in ALPHA:
			model = train_model(res_size)
			model.finalize()
			
			print(f'\n\nTesting with {res_size} nodes, alpha {alpha}:')
			test_results = test_model(model, test_loader)
			print('Test results:')
			# test_acc, test_totals = print_results(test_results)
			# stats[(res_size, alpha)] = (test_acc, test_totals)
			test_scores = print_scores(test_results)
			scores.append(test_scores)
			

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

RuntimeError: t() expects a tensor with <= 2 dimensions, but self is 3D

In [ ]:
boring_scores = [score['boring'] for score in scores]
random_scores = [score['random'] for score in scores]
comp_scores = [score['comp'] for score in scores]
chorales_scores = [score['chorale'] for score in scores]

boring_stats = mean(boring_scores), stdev(boring_scores)
random_stats = mean(random_scores), stdev(random_scores)
comp_stats = mean(comp_scores), stdev(comp_scores)
chorales_stats = mean(chorales_scores), stdev(chorales_scores)

In [ ]:
labels = ['Boring', 'Random', 'Composition', 'Chorales']
x_pos = np.arange(len(labels))
stats = [boring_stats, random_stats, comp_stats, chorales_stats]
means = [stat[0] for stat in stats]
stds = [stat[1] for stat in stats]

# Build the plot
fig, ax = plt.subplots()
ax.bar(x_pos, means, yerr=stds, align='center', alpha=0.5, ecolor='black', capsize=10)
ax.set_ylabel('Score')
ax.set_xticks(x_pos)
ax.set_xticklabels(labels)
ax.set_title('ESN assigned score for chorales and compositions')
# ax.yaxis.grid(True)

# fig.tight_layout()
fig.set_size_inches(15, 10)
plt.savefig('ESN_Scores.jpg', dpi=100)


In [ ]:
labels = ['Boring', 'Random', 'Composition', 'Chorales']
x_pos = np.arange(len(labels))
stats = [boring_stats, random_stats, comp_stats, chorales_stats]
means = [stat[0] for stat in stats]
stds = [stat[1] for stat in stats]

# Build the plot
fig, ax = plt.subplots()
ax.bar(x_pos, means, yerr=stds, align='center', alpha=0.5, ecolor='black', capsize=10)
ax.set_ylabel('Score')
ax.set_xticks(x_pos)
ax.set_xticklabels(labels)
ax.set_title('ESN assigned score for chorales and compositions')
# ax.yaxis.grid(True)

# fig.tight_layout()
fig.set_size_inches(15, 10)
plt.savefig('ESN_Scores.jpg', dpi=100)


In [ ]:
labels = ['Boring', 'Random', 'Composition', 'Chorales']
x_pos = np.arange(len(labels))
stats = [boring_stats, random_stats, comp_stats, chorales_stats]
means = [stat[0] for stat in stats]
stds = [stat[1] for stat in stats]

# Build the plot
fig, ax = plt.subplots()
ax.bar(x_pos, means, yerr=stds, align='center', alpha=0.5, ecolor='black', capsize=10)
ax.set_ylabel('Score')
ax.set_xticks(x_pos)
ax.set_xticklabels(labels)
ax.set_title('ESN assigned score for chorales and compositions')
# ax.yaxis.grid(True)

# fig.tight_layout()
fig.set_size_inches(15, 10)
plt.savefig('ESN_Scores.jpg', dpi=100)
